Q1. What will be the value of dt in the output dataset date? 

In [1]:
DATA date;
    dt = '25dec1959'd;

proc print data = date;
run;

/*Reason for this is 01jan1960 is equal to 1 in SAS system*/

SAS Connection established. Subprocess id is 820



Obs,dt
1,-7


Q2.  Use DATALINES statement to read data directly in the program and create dataset EmpName with two variables Name and Salutation  (2 Marks)

Ram Kumar MR

Neha Venkat MISS

Biplab Pal MR

Priyanka Jain MISS

In [2]:
data EmpName;
input Name $14. Salutation $;
cards;
Ram Kumar      MR
Neha Venkat    MISS
Biplab Pal     MR
Priyanka Jain  MISS

proc print data = EmpName;
run;

/*Remember that input statement does not understand to stop taking names dynamically upon encountering Salutation values.
Put as many spaces as the max length of largest variable in a column in the end of every variable in that column; inversly bring first character
of other column in a vartically straight line to be able to rightly input the data into dataset
*/

Obs,Name,Salutation
1,Ram Kumar,MR
2,Neha Venkat,MISS
3,Biplab Pal,MR
4,Priyanka Jain,MISS


Q3 A) Write a SAS code to create separate dataset for each State without hardcoding statename in the code 

Karnataka   Bangalore

Karnataka   Mysore

Karnataka   Mangalore

WB          Kolkata

WB          Durgapur

Maharashtra Mumbai

Maharashtra Pune

Maharashtra Nasik

In [3]:
data data_set_state;
input state $12. city $9.;
cards;
Karnataka   Bangalore
Karnataka   Mysore
Karnataka   Mangalore
WB          Kolkata
WB          Durgapur
Maharashtra Mumbai
Maharashtra Pune
Maharashtra Nasik
proc print data = data_set_state;
run;

Obs,state,city
1,Karnataka,Bangalore
2,Karnataka,Mysore
3,Karnataka,Mangalore
4,WB,Kolkata
5,WB,Durgapur
6,Maharashtra,Mumbai
7,Maharashtra,Pune
8,Maharashtra,Nasik


In [4]:
proc sql noprint;
select 
        distinct state into:states seperated by ' ' 
        from data_set_state;
quit;

%let list_size =  %sysfunc(countw(&states));

%macro outputting_data(list_size=);
    %put &list_size;
    %do i = 1 %to &list_size;
        %let state = %scan(&states,&i);
        data &state;
            set data_set_state;
            where state = "&state";
            run;
        proc print data = &state;
            run;
    %end;
run;
%mend outputting_data;

%outputting_data(list_size=&list_size);
run;

Obs,state,city
1,Karnataka,Bangalore
2,Karnataka,Mysore
3,Karnataka,Mangalore
Obs,state,city
1,Maharashtra,Mumbai
2,Maharashtra,Pune
3,Maharashtra,Nasik
Obs,state,city
1,WB,Kolkata
2,WB,Durgapur


Q5) Write PROC SQL steps to give bonuses (in points) to frequent flyer program members 

1. 50% bonus who traveled less than 10,000 miles
2. 100% bonus who travelled 10,000 or more

FrequentFlyerID MilesTravelled Points

123458 11983 209

128734 9836 105

129825 50567 734

In [5]:
data frequentflyers;
input FrequentFlyerID MilesTravelled Points;
cards;
123458 11983 209
128734 9836 105
129825 50567 734
run;

proc sql;
    select a.*,
            case when MilesTravelled < 10000 then 1.5*points else 2*Points end as final_points
    from
        frequentflyers as a;
quit;
run;

FrequentFlyerID,MilesTravelled,Points,final_points
123458,11983,209,418
128734,9836,105,157.5
129825,50567,734,1468


Q6)  A Bank wants to send some offers to first 50 premier customers every day if they make online purchases 9:00 AM to 12:00 PM. 
     The base data file (BANK_YYYYMMDD.txt) which has two columns (Customer id and purchase timestamp) is shared by the IT team.

     Write a SAS macro to create a dataset ELIG_CUST that should have the list of eligible customers for the offer; 
     the final dataset ELIG_CUST should have two columns CUST_ID & Name.
1.   Read the cust_id and purchase timestamp from BANK_YYYYDDMM.txt file; Join with CLIENT Dataset to identify premier customers
2.   Exclude customers to whom offers was already sent - Refer to OFFER dataset


In [6]:
/*DATASETS GIVEN TO US*/

data purchase_dataset;
    input customerID purchase_time time8.;
        cards;
        12346 09:05:05
        23456 09:05:07
        34245 12:00:05
        12345 10:11:06
proc print data = purchase_dataset;
run;

data purchase_dataset_hours;
    set purchase_dataset;
        purchase_hours = hour(purchase_time);
proc print data = purchase_dataset_hours;
run;

data client_Table;
input customerID Name $ Segment $;
cards;
12346 Ryan Premier
23456 Victor Gold
34235 Eleena Premier
proc print data = client_Table;
run;

data offers_dataset;
input customerID Offer_Date $9.;
cards;
12346 03DEC2020
23456 05DEC2020
23167 04OCT2020
proc print data = offers_dataset;
run;

/*Final Output*/

/*Here we will get customer ID, NAme as well as Premier segment as well as we exclude customer who already have an offer*/
%macro eligible_customers;
proc sql noprint;
        create table elig_cust 
        as
        select a.customerID,
               Name,
               Segment 
        from  purchase_dataset_hours as a 
        left  join client_Table as b 
               on a.customerID = b.customerID
        where a.customerID not in 
                             (select 
                                    distinct customerID 
                              from offers_dataset
                            ); 
quit;

run;
%mend eligible_customers;

%eligible_customers;
proc print data = elig_cust;
run;

/*Output will not have name and segment as we do not have complete data*/